In [ ]:
from sys import path
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sklearn as sk
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
problem_dir = 'ingestion_program/'  
path.append(problem_dir);
# Uncomment the next lines to auto-reload libraries (this causes some problem with pickles in Python 3)
%load_ext autoreload
%autoreload 2

In [ ]:
 from data_io import read_as_df
data_dir = 'all_data' 
data_name = 'xporters'

In [ ]:
dataFrame = read_as_df(data_dir  + '/' + data_name)  
dataFrame.head()

In [ ]:
data = dataFrame.to_numpy()

In [ ]:
def outlierDetection(data):
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
    outlier = clf.fit_predict(data)
    countInlier = 0
    for i in outlier:
        if i == 1:
            countInlier = countInlier + 1
    realData = np.ndarray(shape=(countInlier,60))
    count = 0
    for i in range(len(outlier)):
        if outlier[i] == 1:
            realData[count] = data[i]
            count = count + 1
    return realData

In [ ]:
def dimensionReduction(nbDimension, data):
    pca = PCA(n_components=nbDimension)
    realData = np.ndarray(shape=(data.shape[0],nbDimension))
    realData = pca.fit_transform(data)
    return realData

In [ ]:
X = outlierDetection(data)

In [ ]:
#X = dimensionReduction(40, X)

In [ ]:
def getTarget(data):
    Y = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        Y[i] = data[i,59]
    return Y

In [ ]:
def featuresSelection(data, dataTarget, optionalEstimator, max_depth_tree, min_sample_leaf_tree):
    clf = ExtraTreesClassifier(n_estimators=optionalEstimator, max_depth=max_depth_tree, min_samples_leaf=min_sample_leaf_tree) 
    clf = clf.fit(data, dataTarget)
    model = SelectFromModel(clf, threshold="mean", prefit=True)
    realData = model.transform(data)
    return realData

In [ ]:
target = getTarget(X)
print(target)

In [ ]:
X = featuresSelection(X_temp, target, 10, 100, 50)
print(X.shape)